In [1]:
import numpy as np, pandas as pd
import seaborn as sns
import RandomCov as rc
import HRP as hrp
import matplotlib.pyplot as mpl
import RCRP as rcrp
%matplotlib inline

In [2]:
# See: https://stackoverflow.com/questions/29351492/how-to-make-a-continuous-alphabetic-list-python-from-a-z-then-from-aa-ab-ac-e
from string import ascii_lowercase
import itertools

def iter_all_strings():
    for size in itertools.count(1):
        for s in itertools.product(ascii_lowercase, repeat=size):
            yield "".join(s)
            
def GetIndexNames(N):
    names = []
    n = 0
    for s in iter_all_strings():
        names.append(s)
        n += 1
        if n >= N:
            return names

In [3]:
# Make a random covariance matrix to run the analysis on
N, k, r, m = 400, 7, 1, 2
np.random.seed(0)
corr   = rc.randomBlockCorr(N, k,random_state=r,minBlockSize=m)
indexNames = GetIndexNames(N)
corr.index, corr.columns = indexNames, indexNames
sigma  = pd.Series( 1. + np.random.rand(N) * 10., index = corr.index )
sharpe = pd.Series( np.random.lognormal(mean=-1./200, sigma = 0.1, size=N), index = corr.index )
#sharpe = pd.Series( np.random.gamma(1., size=N), index = corr.index )
# Make a random set of returns
# Set mu near sigma so that sharpe ratio is near 1 everywhere
mu     = sigma * sharpe
cov    = corr * pd.DataFrame( np.outer(sigma,sigma), index=corr.index, columns=corr.columns )

In [4]:
def stats(w, cov, mu):
    m = np.dot( w , mu )
    s = np.sqrt( np.dot( w.T, np.dot( cov, w ) ) )
    sharpe = m / s
    w2 = w.abs()
    PP = 'mu = ' + str( np.round(m,3) )
    PP += ' sigma = ' + str( np.round( s, 3 ) )
    PP += ' sharpe = ' + str( np.round( sharpe, 3 ) )
    PP += ' sum of weights error = ' + str( np.round( np.abs(1-w.sum()) , 12 ) )
    PP += ' max |w| = ' + str( np.round( w.abs().max() , 3 ) )
    #print 'mu =', np.round(m, 'sigma =', s, 'sharpe =', sharpe , 'sum of weights error =', np.abs(1-w.sum()), 'max |w| =', w.abs().max()
    print PP
    print 'number of large w:', len( w2[w2>w2.mean()+w2.std()*4] )
    return
#------------------------------------------------------------------------------
def Get_HRP_sortIx(corr):
    import scipy.cluster.hierarchy as sch
    dist=hrp.correlDist(corr)
    link=sch.linkage(dist,'single')
    sortIx=hrp.getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist() # recover labels
    return sortIx

In [5]:
print 'min var'
w = pd.Series( np.dot( np.linalg.inv(cov), np.ones(N) ), index = cov.index )
w /= w.sum()
stats( w, cov, mu )
print ''

print 'max sharpe'
w = pd.Series( np.dot( np.linalg.inv(cov), mu ), index = cov.index )
w /= w.sum()
stats( w, cov, mu )
print ''

print 'IVP'
w = pd.Series( hrp.getIVP(cov), index = cov.index )
stats( w, cov, mu )
print ''

print 'IVPNew, extended terms'
w = pd.Series( rcrp.getIVPNew(cov, use_extended_terms=True, a=None), index = cov.index )
stats( w, cov, mu )
print ''

print 'IVPNew, max sharpe'
w = pd.Series( rcrp.getIVPNew(cov, use_extended_terms=False, a=mu), index = cov.index )
stats( w, cov, mu )
print ''

print 'IVPNew, max sharpe, extended terms'
w = pd.Series( rcrp.getIVPNew(cov, use_extended_terms=True, a=mu), index = cov.index )
stats( w, cov, mu )
print ''

sortIx = Get_HRP_sortIx(corr)
print 'HRP'
w = hrp.getRecBipart(cov, sortIx)
stats( w, cov, mu )
print ''

print 'HRPNew'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=False, use_extended_terms2=False, returns=None)
stats( w, cov, mu )
print ''

print 'HRPNew, extended terms1'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=True, use_extended_terms2=False, returns=None)
stats( w, cov, mu )
print ''

print 'HRPNew, extended terms1&2'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=True, use_extended_terms2=True, returns=None)
stats( w, cov, mu )
print ''

print 'HRPNew, max sharpe'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=False, use_extended_terms2=False, returns=mu)
stats( w, cov, mu )
print ''

print 'HRPNew, max sharpe, extended terms1'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=True, use_extended_terms2=False, returns=mu)
stats( w, cov, mu )
print ''

print 'HRPNew, max sharpe, extended terms1&2'
w = rcrp.getRecBipartNew(cov, sortIx, use_extended_terms1=True, use_extended_terms2=True, returns=mu)
stats( w, cov, mu )
print ''

print 'RCRP'
w = rcrp.RCRP( cov )
stats( w, cov, mu )
print ''

print 'RCRP, no extended terms'
w = rcrp.RCRP( cov, use_extended_terms=False )
stats( w, cov, mu )
print ''

print 'RCRP, max sharpe'
w = rcrp.RCRP( cov, a=mu )
stats( w, cov, mu )
print ''

print 'RCRP, no extended terms, max sharpe'
w = rcrp.RCRP( cov, use_extended_terms=False, a=mu )
stats( w, cov, mu )
print ''

min var
mu = -0.033 sigma = 0.169 sharpe = -0.195 sum of weights error = 0.0 max |w| = 0.054
number of large w: 10

max sharpe
mu = -11.095 sigma = 3.104 sharpe = -3.574 sum of weights error = 0.0 max |w| = 0.399
number of large w: 4

IVP
mu = 2.57 sigma = 1.599 sharpe = 1.607 sum of weights error = 0.0 max |w| = 0.025
number of large w: 6

IVPNew, extended terms
mu = -0.02 sigma = 0.22 sharpe = -0.092 sum of weights error = 0.0 max |w| = 0.05
number of large w: 12

IVPNew, max sharpe
mu = 4.147 sigma = 2.524 sharpe = 1.643 sum of weights error = 0.0 max |w| = 0.011
number of large w: 1

IVPNew, max sharpe, extended terms
mu = -20.22 sigma = 6.255 sharpe = -3.233 sum of weights error = 0.0 max |w| = 0.654
number of large w: 3



/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:19: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix


HRP
mu = 5.294 sigma = 3.257 sharpe = 1.625 sum of weights error = 0.0 max |w| = 0.04
number of large w: 7

HRPNew
mu = 5.294 sigma = 3.257 sharpe = 1.625 sum of weights error = 0.0 max |w| = 0.04
number of large w: 7

HRPNew, extended terms1
mu = -1.593 sigma = 13.409 sharpe = -0.119 sum of weights error = 0.0 max |w| = 2.24
number of large w: 4

HRPNew, extended terms1&2
mu = 4.756 sigma = 3.017 sharpe = 1.576 sum of weights error = 0.0 max |w| = 0.069
number of large w: 9

HRPNew, max sharpe
mu = 5.685 sigma = 3.479 sharpe = 1.634 sum of weights error = 0.0 max |w| = 0.017
number of large w: 2

HRPNew, max sharpe, extended terms1
mu = 5.643 sigma = 3.586 sharpe = 1.574 sum of weights error = 0.0 max |w| = 0.044
number of large w: 5

HRPNew, max sharpe, extended terms1&2
mu = 5.808 sigma = 55.096 sharpe = 0.105 sum of weights error = 0.0 max |w| = 1.594
number of large w: 7

RCRP


clusterCorr.py:50: RuntimeWarning: invalid value encountered in double_scalars
  clusterTstats={i:np.mean(silh[ clstrs[i]])/np.std(silh[clstrs[i]]) for i in clstrs.keys()}


mu = 0.226 sigma = 0.259 sharpe = 0.874 sum of weights error = 0.0 max |w| = 0.066
number of large w: 9

RCRP, no extended terms
mu = 2.673 sigma = 1.61 sharpe = 1.66 sum of weights error = 0.0 max |w| = 0.102
number of large w: 5

RCRP, max sharpe
mu = 7.132 sigma = 3.622 sharpe = 1.969 sum of weights error = 0.0 max |w| = 0.215
number of large w: 5

RCRP, no extended terms, max sharpe
mu = 4.861 sigma = 2.872 sharpe = 1.692 sum of weights error = 0.0 max |w| = 0.038
number of large w: 6

